In [1]:
import ee

In [2]:
ee.Authenticate()


True

In [3]:
ee.Initialize()

In [4]:
# Test Earth Engine with a simple command
image = ee.Image('CGIAR/SRTM90_V4')  # A sample dataset
print(image.getInfo())  # Print information about the dataset


{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs': 'EPSG:4326', 'crs_transform': [0.000833333333333, 0, -180, 0, -0.000833333333333, 60]}], 'id': 'CGIAR/SRTM90_V4', 'version': 1641990053291277, 'properties': {'system:visualization_0_min': '-100.0', 'type_name': 'Image', 'keywords': ['cgiar', 'dem', 'elevation', 'geophysical', 'srtm', 'topography'], 'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', 'description': '<p>The Shuttle Radar Topography Mission (SRTM) digital\nelevation dataset was originally produced to provide consistent,\nhigh-quality elevation data at near global scope. This version\nof the SRTM digital elevation data has been processed to fill data\nvoids, and to facilitate its ease of use.</p><p><b>Provider: <a href="https://srtm.csi.cgiar.org/">NASA/CGIAR</a></b><br><p><b>Bands</b><table class="eecat"><tr><th scope="col">Name</th><th

In [5]:
aoi=ee.Geometry.Polygon(
[[39.26196452651823,21.893376873910263],
[39.26283624444807,21.893376873910263],
[39.26283624444807,21.893989109128505],
[39.26196452651823,21.893989109128505],
[39.26196452651823,21.893376873910263]]
)#UJ

In [17]:
aoi=ee.Geometry.Polygon([
 [39.20888129773265,21.8537119026787],
 [39.280979076052965,21.8537119026787],
 [39.280979076052965,21.915198230339893],
 [39.20888129773265,21.915198230339893],
 [39.20888129773265,21.8537119026787]
])

In [ ]:
aoi=ee.Geometry.Polygon([

    [46.60554880056072,24.532105031759635],
    [46.751460970970875,24.455873255767536],
    [46.95333475026775,24.496181878263375],
    [46.942005099388844,24.673822919705184],
    [46.908024931598696,24.723718460355094],
    [46.90217965993572,24.783590439033855],
    [46.917210283767645,24.820537820313316],
    [46.88142308758074,24.87555671889163],
    [46.857204379174,24.960354967272554],
    [46.79987949209019,25.008127706608153],
    [46.74013132009197,25.00967921679065],
    [46.58323282155681,25.046076213753985],
    [46.475370394624484,25.00160657236566],
    [46.373805941674,24.90727431598331],
    [46.40471149070172,24.803726426695388],
    [46.49396890554119,24.644358488808862],
    [46.51286518629248,24.544456555430976],
    [46.586666049095875,24.53915317377673],
    [46.60554880056072,24.532105031759635]

]
)

In [6]:
aoi=ee.Geometry.Polygon( [
[46.41388398001249,24.474133938125608],
[46.99272614309843,24.474133938125608],
[46.99272614309843,25.04372308127158],
[46.41388398001249,25.04372308127158],
[46.41388398001249,24.474133938125608]
])  #new RUH-for ndvi values

In [7]:
import ee
import folium



# Function to add Earth Engine layers to Folium map
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Map Data © Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Map Data © Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.Geometry):    
            folium.GeoJson(
                data=ee_object.getInfo(),
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Map Data © Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
    
    except:
        print("Could not display {}".format(name))

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[21.9, 39.26], zoom_start=12)

# Add the AOI as a layer to the map object.
my_map.add_ee_layer(aoi, {}, 'AOI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)


In [8]:
def maskClouds(image):
    # Example of a simple cloud mask using the QA60 band
    # Actual implementations can be more complex and might use additional bands or indices
    qa60 = image.select('QA60')
    cloudMask = qa60.bitwiseAnd(1 << 10).eq(0)
    return image.updateMask(cloudMask)

In [11]:
# Filter the Sentinel-2 ImageCollection for images within the AOI and time frame, and compute the median
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2024-01-01', '2024-01-17') \
    .filterBounds(aoi) \
    .map(maskClouds) \
    .median()

# Clip the median image to the AOI
clippedImage = sentinelCollection.clip(aoi)

# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Cast all bands of the clipped image to Float32
castedImage = castImage(clippedImage)


 #Export the casted image, specifying scale and region (this is an example, see below for details)
task = ee.batch.Export.image.toDrive(**{
   'image': castedImage,
   'description': 'UJ',
   'scale': 10,
   'region': aoi.getInfo()['coordinates'],
   'fileFormat': 'GeoTIFF',
   'maxPixels': 1e9
 })
task.start()
#Riyadh-all-bands-2024-01-01

In [10]:
# Filter the Sentinel-2 ImageCollection for the specified date range, AOI, and apply cloud masking
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2024-01-01', '2024-01-17') \
    .filterBounds(aoi) \
    .map(maskClouds) \
    .select(['B4', 'B8']) \
    .median()  # Compute the median of the filtered collection

# Clip the median image to the AOI
clippedImage = sentinelCollection.clip(aoi)

# Function to cast selected bands to Float32
def castImage(image):
    return image.toFloat()

# Cast the selected bands of the clipped image to Float32
castedImage = castImage(clippedImage)

# Example export parameters (make sure to replace 'aoi.getInfo()['coordinates']' with your AOI coordinates if necessary)
task = ee.batch.Export.image.toDrive(**{
    'image': castedImage,
    'description': 'Band4_and_Band8_Export',
    'scale': 10,
    'region': aoi.getInfo()['coordinates'],  # Make sure this is correctly defined for your AOI
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e9
})
task.start()

In [11]:
import datetime
# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Function to export images
def exportImage(image, date):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'RUH-B4-B8_{date.strftime("%Y-%m-%d")}',
        scale=10,
        region=aoi.getInfo()['coordinates'],
        fileFormat='GeoTIFF',
        maxPixels=1e9
    )
    task.start()

# Start and end dates
start_date = datetime.datetime(2018, 12, 15)
end_date = datetime.datetime(2024, 1, 30)

# Generate images every 30 days
current_date = start_date
while current_date <= end_date:
    # Define the date range for the current image
    date_range_start = current_date.strftime('%Y-%m-%d')
    next_date = current_date + datetime.timedelta(days=30)
    date_range_end = next_date.strftime('%Y-%m-%d')

    # Get the image collection for the current date range
    sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterDate(date_range_start, date_range_end) \
        .filterBounds(aoi) \
        .map(maskClouds) \
        .select(['B4', 'B8']) \
        .median() 
        
    

    # Clip the image to the AOI
    clippedImage = sentinelCollection.clip(aoi)

    # Cast all bands of the clipped image to Float32
    castedImage = castImage(clippedImage)

    # Export the casted image
    exportImage(castedImage, current_date)

    # Update the current date
    current_date = next_date

print("Images export tasks started.")

Images export tasks started.
